## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 3. Оффлайн-рекомендации

### Постановка задачи

Подготовьте данные для обучения:
- Загрузите ранее сохранённые данные.
- Разбейте данные на тренировочную и тестовую выборки: в тренировочную отнесите все данные до __16 декабря 2022 г. (не включительно)__.

Рассчитайте рекомендации нескольких типов, сохраните каждый тип в своём файле:
- топ популярных — в `top_popular.parquet`,
- персональные (при помощи ALS) — в `personal_als.parquet`,
- похожие треки или i2i (при помощи ALS) — в `similar.parquet`.

Постройте ранжирующую модель, которая использует минимум три признака. Отранжируйте рекомендации, а итоговые сохраните в `recommendations.parquet`.

Проверьте оценку качества трёх типов рекомендаций: 
1. топ популярных,
2. персональных, полученных при помощи ALS,
3. итоговых (по метрикам: recall, precision, coverage, novelty).

Приведите значения метрик в README.

### Инициализация

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score
import scipy
import sklearn.preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from catboost import CatBoostClassifier, Pool

In [2]:
# Проверяем, в каком окружении работаем
print(sys.executable)

/home/mle-user/.venv/bin/python


In [3]:
# Отключаем предупреждения
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning, FutureWarning))

### Загрузка и разбиение данных

In [4]:
# Загружаем основные данные
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

Разбиваем данные на тренировочную и тестовую выборки (в последнюю относим данные строго до 16 декабря 2022 г.)

In [11]:
# Зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16")

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

### Рекомендации на основе самых популярных треков

Загружаем ранее сохраненные топ-треки и оцениваем качество рекомендаций на их основе для "холодных" пользователей.

In [12]:
# Загружаем топ-100 самых прослушиваемых треков
top_k_pop_items = pd.read_parquet("top_popular.parquet")
top_k_pop_items.head()

,rank,item_id,plays,users,name,genres,artists,albums
0,1,53404,15704,15704,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Nirvana],[Smells Like Teen Spirit / In Bloom / On A Pla...
1,2,178529,14088,14088,Numb,"[numetal, metal]",[Linkin Park],"[Meteora, 00s Rock Anthems]"
2,3,37384,12798,12798,Zombie,"[allrock, rock]",[The Cranberries],"[90s Alternative, MNM Sing Your Song: Back To ..."
3,4,6705392,12735,12735,Seven Nation Army,[alternative],[The White Stripes],"[Pay Close Attention : XL Recordings, Radio 1 ..."
4,5,33311009,11972,11972,Believer,"[allrock, rock]",[Imagine Dragons],"[Horoscope Tunes: Cancer, Workout Music Hits 2..."


In [13]:
# Идентификаторы уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

In [14]:
# Идентификаторы "холодных" пользователей
cold_users = users_test[~users_test.isin(users_train)]
print(f"Кол-во холодных пользователей: {len(cold_users)}")

Кол-во холодных пользователей: 0


Удаление большого кол-ва записей из истории взаимодействий привело к тому, что холодных пользователей не оказалось. Из-за этого оценить показатели рекомендаций по умолчанию не получится.

Ниже показан код для расчета точности и покрытия, он выдает пустые значения из-за отсутствия холодных пользователей.

In [15]:
# Формируем бинарные колонки target и prediction

top_k_pop_items['prediction'] = 1

cold_users_events = \
    events_test[events_test["user_id"].isin(cold_users)] \
        .merge(top_k_pop_items[['item_id', 'prediction']], on="item_id", how="left")

cold_users_events['prediction'].fillna(0, inplace=True)

cold_users_events['target'] = 1

In [16]:
# Точность рекомендаций
accuracy = accuracy_score(cold_users_events['prediction'], cold_users_events['target'])
print(accuracy)

nan


/home/mle-user/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mle-user/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [17]:
# Считаем покрытие "холодных" пользователей рекомендациями.
cold_users_hit_ratio = cold_users_events.groupby("user_id").agg(hits=("prediction", 'sum'))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: nan
Среднее покрытие пользователей: nan


### Персональные рекомендации на основе коллаборативного подхода и ALS

[Техническая документация на библиотеку implicit](https://benfred.github.io/implicit/api/models/cpu/als.html)


Перекодируем идентификаторы `user_id` и `item_id`.

In [18]:
# Перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train = events_train.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left')
events_test = events_test.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left')

Создаем sparse-матрицу в формате [CSR](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [19]:
# Формируем колонку target (1 - трек прослушан, 0 - нет)
events_train['target'] = 1

# Создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["target"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

Имея подготовленную матрицу взаимодействий, создаем ALS-модель. Для примера возьмём количество латентных факторов для матриц $P и Q$ равным 50. 

In [20]:
# Код для создания и тренировки модели
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/.venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [21]:
# Сохраняем als-модель в файл
os.makedirs('models/', exist_ok=True)
with open('models/als_model.pkl', 'wb') as fd:
    joblib.dump(als_model, fd)

Чтобы получить рекомендации для отдельного пользователя с помощью модели ALS, используем функцию ниже.

In [22]:
def get_recommendations_als(user_item_matrix, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = als_model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Посмотрим историю и рекомендации для случайного пользователя.

In [23]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]
print(f"user_id: {user_id}")

print("Последние события:")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации:")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=10)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
user_recommendations_to_print = \
    user_recommendations[["item_id", "name", "genres", "albums", "artists", "score"]].head(10)
display(user_recommendations_to_print)

user_id: 942522
Последние события:


,item_id,name,genres,albums,artists
4242,87550631,Вы как там,"[rusrap, rap]",[К ЧАЮ],[Жаман]
4243,87550638,Лети,"[rusrap, rap]",[К ЧАЮ],"[Алина Маслова, Жаман]"
4244,87557144,Megamix,"[pop, ruspop]",[Megamix],[Ангина]
4245,87563076,Babel,[dance],[Babel],[Gustavo Bravetty]
4246,87596860,DO OR DIE,"[rap, foreignrap]",[DO OR DIE],[Dxrk ダーク]
4247,87597410,Nico Nii,[electronics],[Nico Nii],[Leo Motoko]
4248,87647751,Breathe,[electronics],[Breathe],"[The Prodigy, Rene Lavice, RZA]"
4249,87739115,Белые туманы,"[pop, ruspop]",[Белые туманы],[ГУДЗОН]
4250,87830870,Let The Monsters Free,[dance],[MY SPIRIT (2016)],"[Droplex, FiveAm]"
4251,87831236,forgotten,[electronics],[forgotten],[$werve]


Рекомендации:


,item_id,name,genres,albums,artists,score
0,48591611,Урбан,"[rusrap, rap]",[Баста 3],[Баста],1.393190
1,68348390,Там ревели горы,"[rusrap, rap]",[YAMAKASI],[Miyagi & Andy Panda],1.286180
2,29175370,Рапапам,"[rusrap, rap]",[Рапапам],"[9 грамм, Miyagi & Эндшпиль]",1.239922
3,65320578,Brooklyn,"[rusrap, rap]",[Brooklyn],"[Miyagi & Andy Panda, TumaniYO]",1.229075
4,68348389,Minor,"[rusrap, rap]",[YAMAKASI],[Miyagi & Andy Panda],1.215466
5,48592207,"Тем, кто с нами","[rusrap, rap]",[Мегаpolice],"[Ноггано, GUF, АК-47]",1.196820
6,48591644,Солнца не видно,"[rusrap, rap]",[Баста 3],"[Баста, Бумбокс]",1.171954
7,53729475,Marlboro,"[rusrap, rap]",[Buster Keaton],[Miyagi],1.171645
8,47354783,Джанго,"[rusrap, rap]","[Неизданное, Джанго, Неизданное. Part 1]",[Эндшпиль],1.074131
9,68348391,Yamakasi,"[rusrap, rap]",[YAMAKASI],[Miyagi & Andy Panda],1.071803


Видим, что рекомендации хорошо мэтчатся с историей пользователя.

Получим рекомендации для всех имеющихся пользователей.

In [24]:
# Получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# Получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, 
    N=100)

Код возвращает рекомендации как список списков, что не очень удобно. Преобразуем его в более удобный табличный формат.

In [25]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [26]:
# Смотрим, что получилось
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.head()

,user_id,item_id,score
0,16,629150,1.201058
1,16,629743,1.198524
2,16,17179,1.178191
3,16,208740,1.156959
4,16,22436,1.147422


In [27]:
# Сохраним полученные рекомендации в файл
als_recommendations.to_parquet("personal_als.parquet")

Найдем покрытие по объектам, используя все имеющиеся рекомендации.

In [30]:
cov_items = als_recommendations['item_id'].nunique() / len(items) 
print(cov_items)

0.010876911487221414


Посчитаем среднее Novelty@5, для этого:
- разметим каждую рекомендацию в als_recommendations булевым признаком played, используя events_train
(False — пользователь не прослушал трек, True — прослушал),
- посчитаем Novelty@5 для каждого пользователя,
- найдем среднее значение полученных Novelty@5.

In [31]:
# разметим каждую рекомендацию признаком played
events_train["played"] = True
als_recommendations = als_recommendations \
    .merge(events_train[['played', "user_id", "item_id"]], on=["user_id", "item_id"], how="left")
als_recommendations["played"] = als_recommendations["played"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by='score', ascending=False) 
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["played"].mean())

# посчитаем средний novelty
novelty_5_mean = novelty_5.mean()
print(novelty_5_mean)

0.21277466417578508


Посчитаем recall и precision для ALS-рекомендаций. Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
- gt (ground truth): объект есть в тестовой выборке;
- pr (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
- TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
- FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
- FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

Создаем функцию для подсчета метрик TP, FP и FN у пользователей, которые есть одновременно и в events_train,
и в events_test.

In [32]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

Считаем TP, FP и FN для каждой пары user_id, item_id

In [33]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(events_train,
                                                                        events_test, 
                                                                        als_recommendations,
                                                                        top_k=5)

Common users: 24210


Считаем precision и recall

In [34]:
def compute_cls_metrics(events_recs_for_binary_metrics):
    
    groupper = events_recs_for_binary_metrics.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum()) 
    recall = recall.fillna(0).mean()

    return precision, recall

In [35]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
print(precision, recall)

0.00047914085088806283 2.664149516353011e-05


### Рекомендации на основе контентного подхода и ALS

Построим контентные рекомендации на основе принадлежности треков к различным жанрам.

Сначала загрузим список жанров по всем трекам, который мы создали на предыдущем этапе

In [36]:
genres = pd.read_parquet('genres.parquet')
genres.head()

,name,items_count,score
genre_id,,,
0,pop,166284,0.103221
1,folk,37547,0.023307
2,allrock,118506,0.073563
3,hardrock,10916,0.006776
4,rock,55174,0.034249


Функция ниже строит матрицу вида «трек-жанр»

In [38]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genres"] is None: # После предобработки NA быть не может, оставили для общего случая
            continue
        
        item_genres = list(v["genres"])
        
        for genre_name in item_genres:
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(1)
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), \
                                         shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанрам была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

Построим матрицу с весами по жанрам для каждого трека

In [39]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Будем искать рекомендации на основе косинусной близости. Функция ниже находит рекомендации для одного пользователя

In [40]:
def get_recommendations_by_genres(user_id, items, events, all_items_genres_csr, n=5):

    """ Возвращает n рекомендаций для user_id на основе предпочитаемых им жанров"""
    
    user_events = events.query("user_id == @user_id")
    user_items = items[items["item_id"].isin(user_events["item_id"])]

    user_items_genres_csr = all_items_genres_csr[user_items['item_id_enc']]
    
    # вычислим среднюю склонность пользователя к жанрам 
    user_genres_scores = np.asarray(user_items_genres_csr.mean(axis=0))

    # вычисляем сходство между треками пользователя и всеми имеющимися трекам
    similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

    # преобразуем в одномерный массив
    similarity_scores = similarity_scores.flatten()

    # получаем индексы top-n (по убыванию значений), т.е. encoded индексы самых похожих треков
    top_n_indices = np.argsort(-similarity_scores)[:n] 

    selected_items = items[items["item_id_enc"].isin(top_n_indices)]
    
    return selected_items['item_id'].to_list(), similarity_scores[top_n_indices].tolist()

Посмотрим рекомендации для произвольного пользователя

In [41]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]
print(f"user_id: {user_id}")

print("Последние события:")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации:")
selected_items, similarity_scores = \
    get_recommendations_by_genres(user_id, items, events_train, all_items_genres_csr, n=5)
selected_items = items[items["item_id"].isin(selected_items)]
selected_items['scores'] = similarity_scores

with pd.option_context("max_colwidth", 100):
   display(selected_items[["item_id", "name", "genres", "albums", "artists", 'scores']])

user_id: 314986
Последние события:


,item_id,name,genres,albums,artists
1480,99030871,Myriad Wave,[electronics],[Myriad Wave],[NBSPLV]
1481,99233920,Close Your Eyes,"[electronics, dnb]","[Close Your Eyes, Cognition]","[Wilkinson, iiola]"
1482,99236464,Whisk It Up,"[rap, foreignrap]",[Cake and Cognac],"[Dillon Francis, Tommy Cash, Yung Gravy]"
1483,99237151,"Chronicles of a Space Man on Planet Earth, Vol. 4",[electronics],"[Chronicles of a Space Man on Planet Earth, Vo...",[The Polish Ambassador]
1484,99363737,The Smoke,[indie],[The Smoke],[The Smile]
1485,99487535,Morning,"[electronics, house]",[Morning],"[Kely$e, Corrupt (UK)]"
1486,99929653,Substantial,[electronics],[Substantial],[NBSPLV]
1487,100387548,Eyes Alive,[electronics],[Eyes Alive],[HVOB]
1488,100588267,The Actor,[indie],"[The Actor, The Dream]",[alt-J]
1489,100715572,When It Rains,"[rap, foreignrap]",[When It Rains],"[BackRoad Gee, Chase & Status]"


Рекомендации:


,item_id,name,genres,albums,artists,scores
777677,61107512,Burn Me Down,[electronics],[Burn Me Down],[Sekai],0.925659
777678,61108170,Power Is Taken,[electronics],[Power Is Taken],[Moby],0.925659
777714,61113957,Weeble Wobble,[electronics],[Weeble Wobble EP],[Barely Alive],0.925659
869223,72763830,Uh La La,[electronics],[Uh La La],[XIX],0.925659
869260,72766686,Work 2.0,[electronics],"[Work 2.0, Viral Hits TikTok 5]","[Shinnas Way, Eduardo Luzquiños]",0.925659


Видим, что рекомендации хорошо мэтчатся с историей пользователя.

Запускаем поиск для пользователей из events_train. Для экономии ресурсов возьмем ограниченное кол-во user_id, причем представленных как в events_train, так и в `als_recommendations`, чтобы в датасете для ранжирующей модели было много строк, где обе оценки scores непустые.

In [53]:
# Берем ограниченное кол-во юзеров, содержащихся одновременно в events_train и als_recommendations
users_num = 10000
users_ids = \
    als_recommendations[als_recommendations['user_id'].isin(events_train['user_id'].unique())]['user_id'][:users_num] 

In [54]:
# Считаем контентные рекомендации для всех пользователей из users_ids

n = 5
items_ids = []
scores = []
for i, user_id in tqdm(enumerate(users_ids), total=users_num):
    user_items_ids, user_scores = \
        get_recommendations_by_genres(user_id, items, events_train, all_items_genres_csr, n)
    items_ids.append(user_items_ids)
    scores.append(user_scores)

content_recommendations = pd.DataFrame({
    "user_id": users_ids,
    "item_id": items_ids, 
    "score": scores})

content_recommendations = content_recommendations.explode(["item_id", "score"], ignore_index=True)

100%|██████████| 10000/10000 [1:12:00<00:00,  2.31it/s]


In [55]:
# Смотрим результат
content_recommendations

,user_id,item_id,score
0,801589,512943,0.851419
1,801589,14685651,0.851419
2,801589,14685655,0.851419
3,801589,31109247,0.851419
4,801589,32423679,0.851419
...,...,...,...
49995,1134659,73495,0.759595
49996,1134659,29047803,0.759595
49997,1134659,31109254,0.759595
49998,1134659,31109255,0.759595


In [56]:
# Сохраним полученные рекомендации в файл
content_recommendations.to_parquet("content_recommendations.parquet")

### Рекомендации на основе похожих треков (i2i) и ALS

Получим рекомендации при помощи метода [similar_items](https://benfred.github.io/implicit/api/models/cpu/als.html). Для экономии ресурсов возьмем не все объекты из `events_train`, а только часть.

In [ ]:
# Загружаем ALS-модель
with open('models/als_model.pkl', 'rb') as fd:
    als_model = joblib.load(fd)

In [60]:
# Кол-во объектов из events_train, для которых будем искать похожие треки
items_num = 100000

In [61]:
# Получим энкодированные идентификаторы объектов в events_train
train_item_ids_enc = events_train['item_id_enc'].unique()[:items_num]

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()}) 
similar_items = similar_items.explode(['sim_item_id_enc', 'score'], ignore_index=True) 

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype('int') 
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"]) 
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"]) 
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [62]:
# Смотрим результат
similar_items.head()

,score,item_id_1,item_id_2
1,0.924673,549,26771
2,0.923671,549,4198
3,0.921660,549,69658
4,0.921044,549,37402640
5,0.913545,549,1706463


In [66]:
# Сохраняем полученные рекомендации в файл
similar_items.to_parquet("similar.parquet")

С помощью функции ниже можно посмотреть рекомендации для одного трека

In [67]:
def print_sim_items(item_id, similar_items):
    item_columns_to_use = ["item_id", "name", "genres", "artists", "albums"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [68]:
# Задаем произвольный трек
item_id = similar_items['item_id_1'].sample().iat[0]

# Смотрим похожие треки
print_sim_items(item_id, similar_items)

,item_id,name,genres,artists,albums
300930,19528114,Fire And Ice,[folk],[Ernesto Cortazar],[Forever You and I]


,score,item_id_1,item_id_2,name,genres,artists,albums
979012,0.989849,19528114,19528085,Autumn Rose,[folk],[Ernesto Cortazar],[Just the Two of Us]
979013,0.989584,19528114,19534491,What Happened Between Us,"[modern, classicalmusic]",[Ernesto Cortazar],[Just for You]
979014,0.988714,19528114,19533839,So Sad To Say Goodbye,"[modern, classicalmusic]",[Ernesto Cortazar],[On the Top of the World]
979015,0.985689,19528114,729428,Corazón de Niño,"[pop, latinfolk, folk]",[Raúl Di Blasio],"[Mis Favoritas, La Historia Del Piano De Améri..."
979016,0.985667,19528114,19534489,L'adieu,"[modern, classicalmusic]",[Ernesto Cortazar],[Just for You]
979017,0.985329,19528114,20723120,Renaissance,"[relax, newage]",[Giovanni],[Romantico]
979018,0.985329,19528114,19528019,Legend Of The Sea,"[modern, classicalmusic]",[Ernesto Cortazar],[Legend of the Sea]
979019,0.984248,19528114,7450179,Barroco,[pop],[Raúl Di Blasio],[Las No. 1 Instrumentales]
979020,0.984242,19528114,19528030,Foolish Heart,"[modern, classicalmusic]",[Ernesto Cortazar],[60 Years]
979021,0.983760,19528114,19528017,Secrets Of My Heart,"[modern, classicalmusic]",[Ernesto Cortazar],[Legend of the Sea]


Видим, что найденные объекты действительно похожи на заданный.

### Построение ранжирующей модели

Подготовим список кандидатов для обучения ранжирующей модели. В качестве кандидатогенераторов возьмём ранее подготовленные при помощи ALS коллаборативные и контентные рекомендации. Кроме того, добавим 3-й признак - кол-во треков, прослушанных каждым пользователем.

In [6]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("personal_als.parquet")
content_recommendations = pd.read_parquet("content_recommendations.parquet")

In [69]:
# Объединяем списки рекомендаций по совпадению user_id, item_id
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer") 

Оставляем только те строки, где есть оценки от обоих кандидатогенераторов

In [70]:
candidates = candidates[~candidates[['als_score', 'cnt_score']].isnull().any(axis=1)]
candidates

,user_id,item_id,als_score,cnt_score
15457,920932,73495,1.501268,0.916823
15458,920932,73495,1.501268,0.916823
15459,920932,73495,1.501268,0.916823
15460,920932,73495,1.501268,0.916823
20074,947040,670578,1.455110,0.786586
...,...,...,...,...
115465,506335,27317895,1.113865,0.905386
115466,506335,27317895,1.113865,0.905386
140013,650552,34193879,1.073689,0.899798
152413,1021081,41473,1.055705,0.621384


Добавим в candidates таргеты, используя историю взаимодействий в events_train:
- 1 - для прослушанных item_id (положительный пример),
- 0 - для всех остальных (негативный пример).

В candidates отберем все положительные примеры, а также не менее четырёх негативных для каждого пользователя в положительных примерах.

In [71]:
# Добавляем таргет к кандидатам
events_train["target"] = 1
candidates = candidates.merge(events_train[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], 
                              how='left') 
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
#candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

candidates_for_train = candidates
'''
# для каждого пользователя оставляем только 2 негативных примера
negatives_per_user = 2
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"), 
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])
'''

'\n# для каждого пользователя оставляем только 2 негативных примера\nnegatives_per_user = 2\ncandidates_for_train = pd.concat([\n    candidates_to_sample.query("target == 1"), \n    candidates_to_sample.query("target == 0")         .groupby("user_id")         .apply(lambda x: x.sample(negatives_per_user, random_state=0))\n    ])\n'

Добавим в events_train новый признак пользователей - количество треков, прослушанных каждым пользователем

In [72]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        tracks_played_by_user =("started_at", "count")
    )

    return user_features
    

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
candidates_for_train

,user_id,item_id,als_score,cnt_score,target,tracks_played_by_user
0,920932,73495,1.501268,0.916823,1,3753
1,920932,73495,1.501268,0.916823,1,3753
2,920932,73495,1.501268,0.916823,1,3753
3,920932,73495,1.501268,0.916823,1,3753
4,947040,670578,1.455110,0.786586,1,4574
...,...,...,...,...,...,...
90,506335,27317895,1.113865,0.905386,1,3743
91,506335,27317895,1.113865,0.905386,1,3743
92,650552,34193879,1.073689,0.899798,1,6295
93,1021081,41473,1.055705,0.621384,1,3894


Обучаем модель

In [73]:
# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 'tracks_played_by_user']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.5178265	total: 52.7ms	remaining: 52.7s
100:	learn: 0.0035319	total: 87.4ms	remaining: 778ms
200:	learn: 0.0021320	total: 122ms	remaining: 483ms
300:	learn: 0.0021320	total: 154ms	remaining: 357ms
400:	learn: 0.0021320	total: 186ms	remaining: 278ms
500:	learn: 0.0021320	total: 218ms	remaining: 217ms
600:	learn: 0.0021320	total: 250ms	remaining: 166ms
700:	learn: 0.0021320	total: 282ms	remaining: 120ms
800:	learn: 0.0021320	total: 315ms	remaining: 78.3ms
900:	learn: 0.0021320	total: 348ms	remaining: 38.2ms
999:	learn: 0.0021320	total: 378ms	remaining: 0us


In [74]:
# Сохраняем ранжирующую модель
os.makedirs('models/', exist_ok=True)
cb_model.save_model('models/cb_model.cbm')

Теперь можно получить финальные рекомендации, используя обученную модель.

In [75]:
# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_for_train[candidates_for_train["user_id"].isin(events_test["user_id"].drop_duplicates())]

inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id').cumcount() + 1 

max_recommendations_per_user = 100
recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user') \
    .rename(columns={"cb_score": "score"}) \
        .drop(columns='target')

In [76]:
# Смотрим результат
recommendations

,user_id,item_id,als_score,cnt_score,tracks_played_by_user,score,rank
79,12014,628383,1.225516,0.791916,4526,0.996331,1
80,12014,628383,1.225516,0.791916,4526,0.996331,2
81,12014,628383,1.225516,0.791916,4526,0.996331,3
82,12014,628383,1.225516,0.791916,4526,0.996331,4
83,12014,628383,1.225516,0.791916,4526,0.996331,5
...,...,...,...,...,...,...,...
72,1196792,43013827,1.286927,0.88068,6771,0.999468,16
73,1196792,43013827,1.286927,0.88068,6771,0.999468,17
74,1196792,43013827,1.286927,0.88068,6771,0.999468,18
75,1196792,43013827,1.286927,0.88068,6771,0.999468,19


In [77]:
# Сохраняем финальные персональные рекомендации в файл
recommendations.to_parquet("recommendations.parquet")

Проведем валидацию финальных рекомендаций.

Посчитаем метрики recall и precision, используя отложенную тестовую выборку, 
а также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [78]:
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_train,
    events_test, 
    recommendations, 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5) 

Common users: 16


In [79]:
print(f"precision: {cb_precision_5}, recall: {cb_recall_5}")

precision: 0.0, recall: 0.0


Низкий результат обусловлен ограниченным размером обучающей выборки.

Посмотрим, какие признаки вносят наибольший вклад в ранжирование. 

In [80]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])

feature_importance = feature_importance.sort_values('fi', ascending=False)
feature_importance

,fi
tracks_played_by_user,49.148459
cnt_score,42.319047
als_score,8.532494
